In [1]:
import requests # requests機能をインポートします。
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13109&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1'
res = requests.get(url) #リクエストしたデータをresに代入します。
result = res.text #resのtextにサイトデータが格納されているので、それをresultに代入します。

In [2]:
print(result) # printでサイトデータを確認します。

<!DOCTYPE html>
<!--[if lt IE 7]><html lang="ja" class="ie"><![endif]-->
<!--[if IE 7]><html lang="ja" class="ie"><![endif]-->
<!--[if IE 8]><html lang="ja" class="ie"><![endif]-->
<!--[if gt IE 8]><!--><html lang="ja"><!--<![endif]-->
<!-- FR301FC0012 START -->
<head>
<meta charset="utf-8" />
<!-- seo.headerTag Start -->

<title>【SUUMO】東京都品川区の賃貸(賃貸マンション・アパート)住宅のお部屋探し物件情報</title>

<meta name="keywords" content='品川区,東京都,賃貸,賃貸マンション,アパート,賃貸アパート,賃貸情報,部屋探し,賃貸住宅,物件,賃貸物件,住まい,SUUMO,スーモ' />

<meta name="description" content='【SUUMO(スーモ)賃貸】東京都品川区でアパート・賃貸マンションを借りるならリクルートの賃貸情報サイトSUUMO(スーモ)にお任せください。全国の賃貸住宅・貸家の物件情報からさまざまな条件を指定して検索！豊富な賃貸物件情報からあなたにピッタリの住まい情報を見つけてください。' />

<meta name="robots" content="noindex, follow" />
<link rel="shortcut icon" href="/front/img/favicon.ico" type="image/vnd.microsoft.icon" />
<link rel="icon" href="/front/img/favicon.ico" type="image/vnd.microsoft.icon" />

<meta name="robots" content="noarchive">
<!-- seo.headerTag End -->
<link rel="stylesheet" href="/edit/assets/s

In [3]:
print(res.encoding) # res.encodingで文字エンコードを確認できます。

UTF-8


In [4]:
from bs4 import BeautifulSoup # BeautifulSoupの機能をインポートします。
soup = BeautifulSoup(res.text,"html.parser") # BeautifulSoup(解析したいデータ,解析する方法)を指定し、解析したデータをsoupに代入します。

In [5]:
from retry import retry
import urllib
import time

In [6]:
# 複数ページの情報をまとめて取得
data_samples = []

# スクレイピングするページ数
max_page = 10

# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

In [7]:
# ページごとの処理
for page in range(1,max_page+1):
    before = time.time()
    # ページ情報
    soup = load_page(url.format(page))
    # 物件情報リストを指定
    mother = soup.find_all(class_='cassetteitem')

    # 物件ごとの処理
    for child in mother:
        # 建物情報
        data_home = []
        # カテゴリ
        data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
        # 建物名
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        # 住所
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        # 最寄り駅のアクセス
        children = child.find(class_='cassetteitem_detail-col2')
        for id,grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
            data_home.append(grandchild.text)
        # 築年数と階数
        children = child.find(class_='cassetteitem_detail-col3')
        for grandchild in children.find_all('div'):
            data_home.append(grandchild.text)
            # 部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []

            # 部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                # 階
                if id_ == 2:
                    data_room.append(grandchild.text.strip())
                # 家賃と管理費
                elif id_ == 3:
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                # 敷金と礼金
                elif id_ == 4:
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                # 間取りと面積
                elif id_ == 5:
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
                # url
                elif id_ == 8:
                    get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    abs_url = urllib.parse.urljoin(url,get_url)
                    data_room.append(abs_url)
            # 物件情報と部屋情報をくっつける
            data_sample = data_home + data_room
            data_samples.append(data_sample)

    # 1アクセスごとに1秒休む
    time.sleep(1)


In [8]:
data_samples

[['賃貸マンション',
  'パークシティ大崎\u3000ザ\u3000タワー',
  '東京都品川区北品川５',
  'ＪＲ山手線/大崎駅 歩8分',
  '東急池上線/五反田駅 歩12分',
  '京急本線/北品川駅 歩17分',
  '築9年',
  '地下2地上40階建',
  '11階',
  '24.7万円',
  '10000円',
  '24.7万円',
  '24.7万円',
  '1LDK',
  '43.77m2',
  'https://suumo.jp/chintai/jnc_000086103920/?bc=100351315698'],
 ['賃貸マンション',
  'パークシティ大崎\u3000ザ\u3000タワー',
  '東京都品川区北品川５',
  'ＪＲ山手線/大崎駅 歩8分',
  '東急池上線/五反田駅 歩12分',
  '京急本線/北品川駅 歩17分',
  '築9年',
  '地下2地上40階建',
  '6階',
  '28.9万円',
  '10000円',
  '28.9万円',
  '57.8万円',
  '2LDK',
  '54.06m2',
  'https://suumo.jp/chintai/jnc_000085009879/?bc=100349209702'],
 ['賃貸マンション',
  'パークシティ大崎\u3000ザ\u3000タワー',
  '東京都品川区北品川５',
  'ＪＲ山手線/大崎駅 歩8分',
  '東急池上線/五反田駅 歩12分',
  '京急本線/北品川駅 歩17分',
  '築9年',
  '地下2地上40階建',
  '5階',
  '31.3万円',
  '10000円',
  '31.3万円',
  '62.6万円',
  '2LDK',
  '57.57m2',
  'https://suumo.jp/chintai/jnc_000085347303/?bc=100257255178'],
 ['賃貸マンション',
  'パークシティ大崎\u3000ザ\u3000タワー',
  '東京都品川区北品川５',
  'ＪＲ山手線/大崎駅 歩8分',
  '東急池上線/五反田駅 歩12分',
  '京急本線/北品川駅 歩17分',
  '築9年',
  '地下2地上40

In [9]:
import pandas as pd
# 表示オプションをデフォルトに戻す
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
df = pd.DataFrame(data_samples)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,11階,24.7万円,10000円,24.7万円,24.7万円,1LDK,43.77m2,https://suumo.jp/chintai/jnc_000086103920/?bc=...
1,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,6階,28.9万円,10000円,28.9万円,57.8万円,2LDK,54.06m2,https://suumo.jp/chintai/jnc_000085009879/?bc=...
2,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,5階,31.3万円,10000円,31.3万円,62.6万円,2LDK,57.57m2,https://suumo.jp/chintai/jnc_000085347303/?bc=...
3,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,16階,37.5万円,20000円,75万円,37.5万円,2SLDK,67.05m2,https://suumo.jp/chintai/jnc_000085227510/?bc=...
4,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,35階,43万円,10000円,43万円,43万円,2LDK,81.27m2,https://suumo.jp/chintai/jnc_000080487072/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2375,賃貸アパート,プリモパッソ,東京都品川区東大井６,ＪＲ京浜東北線/大井町駅 歩6分,京急本線/立会川駅 歩11分,京急本線/鮫洲駅 歩11分,新築,3階建,2階,9.7万円,4000円,9.7万円,19.4万円,1K,27.08m2,https://suumo.jp/chintai/jnc_000086708919/?bc=...
2376,賃貸アパート,プリモパッソ,東京都品川区東大井６,ＪＲ京浜東北線/大井町駅 歩6分,京急本線/立会川駅 歩11分,京急本線/鮫洲駅 歩11分,新築,3階建,3階,9.9万円,4000円,9.9万円,19.8万円,1K,28.26m2,https://suumo.jp/chintai/jnc_000086672177/?bc=...
2377,賃貸アパート,プリモパッソ,東京都品川区東大井６,ＪＲ京浜東北線/大井町駅 歩6分,京急本線/立会川駅 歩11分,京急本線/鮫洲駅 歩11分,新築,3階建,3階,9.9万円,4000円,9.9万円,19.8万円,1K,28.26m2,https://suumo.jp/chintai/jnc_000086708920/?bc=...
2378,賃貸マンション,Log品川サーフサイド,東京都品川区南大井３,京急本線/大森海岸駅 歩6分,ＪＲ京浜東北線/大森駅 歩6分,ＪＲ京浜東北線/大井町駅 歩29分,築7年,12階建,4階,9.2万円,10000円,-,-,1K,20.63m2,https://suumo.jp/chintai/jnc_000086561568/?bc=...


In [10]:
df = df.rename(columns = {0:'カテゴリ',1:'建物名',2:'住所',3:'最寄り駅のアクセス1',4:'最寄り駅のアクセス2',5:'最寄り駅のアクセス3',6:'築年数',7:'階数情報',8:'物件階数',9:'家賃',10:'管理費',11:'礼金',12:'敷金',13:'間取り',14:'面積',15:'URL'})
df

,カテゴリ,建物名,住所,最寄り駅のアクセス1,最寄り駅のアクセス2,最寄り駅のアクセス3,築年数,階数情報,物件階数,家賃,管理費,礼金,敷金,間取り,面積,URL
0,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,11階,24.7万円,10000円,24.7万円,24.7万円,1LDK,43.77m2,https://suumo.jp/chintai/jnc_000086103920/?bc=...
1,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,6階,28.9万円,10000円,28.9万円,57.8万円,2LDK,54.06m2,https://suumo.jp/chintai/jnc_000085009879/?bc=...
2,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,5階,31.3万円,10000円,31.3万円,62.6万円,2LDK,57.57m2,https://suumo.jp/chintai/jnc_000085347303/?bc=...
3,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,16階,37.5万円,20000円,75万円,37.5万円,2SLDK,67.05m2,https://suumo.jp/chintai/jnc_000085227510/?bc=...
4,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,35階,43万円,10000円,43万円,43万円,2LDK,81.27m2,https://suumo.jp/chintai/jnc_000080487072/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2375,賃貸アパート,プリモパッソ,東京都品川区東大井６,ＪＲ京浜東北線/大井町駅 歩6分,京急本線/立会川駅 歩11分,京急本線/鮫洲駅 歩11分,新築,3階建,2階,9.7万円,4000円,9.7万円,19.4万円,1K,27.08m2,https://suumo.jp/chintai/jnc_000086708919/?bc=...
2376,賃貸アパート,プリモパッソ,東京都品川区東大井６,ＪＲ京浜東北線/大井町駅 歩6分,京急本線/立会川駅 歩11分,京急本線/鮫洲駅 歩11分,新築,3階建,3階,9.9万円,4000円,9.9万円,19.8万円,1K,28.26m2,https://suumo.jp/chintai/jnc_000086672177/?bc=...
2377,賃貸アパート,プリモパッソ,東京都品川区東大井６,ＪＲ京浜東北線/大井町駅 歩6分,京急本線/立会川駅 歩11分,京急本線/鮫洲駅 歩11分,新築,3階建,3階,9.9万円,4000円,9.9万円,19.8万円,1K,28.26m2,https://suumo.jp/chintai/jnc_000086708920/?bc=...
2378,賃貸マンション,Log品川サーフサイド,東京都品川区南大井３,京急本線/大森海岸駅 歩6分,ＪＲ京浜東北線/大森駅 歩6分,ＪＲ京浜東北線/大井町駅 歩29分,築7年,12階建,4階,9.2万円,10000円,-,-,1K,20.63m2,https://suumo.jp/chintai/jnc_000086561568/?bc=...


In [11]:
# duplicated()で重複がしているかをTrueかFalseで返してくれます。
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2375     True
2376     True
2377     True
2378     True
2379     True
Length: 2380, dtype: bool

In [12]:
# drop_duplicates()で重複したデータを削除してくれます。
# drop_duplicates(inplace=True)とすることで、処理したデータを出力だけでなく、出力したデータを元のdfに代入して変更してくれます。
df.drop_duplicates(inplace=True)
df

,カテゴリ,建物名,住所,最寄り駅のアクセス1,最寄り駅のアクセス2,最寄り駅のアクセス3,築年数,階数情報,物件階数,家賃,管理費,礼金,敷金,間取り,面積,URL
0,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,11階,24.7万円,10000円,24.7万円,24.7万円,1LDK,43.77m2,https://suumo.jp/chintai/jnc_000086103920/?bc=...
1,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,6階,28.9万円,10000円,28.9万円,57.8万円,2LDK,54.06m2,https://suumo.jp/chintai/jnc_000085009879/?bc=...
2,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,5階,31.3万円,10000円,31.3万円,62.6万円,2LDK,57.57m2,https://suumo.jp/chintai/jnc_000085347303/?bc=...
3,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,16階,37.5万円,20000円,75万円,37.5万円,2SLDK,67.05m2,https://suumo.jp/chintai/jnc_000085227510/?bc=...
4,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,35階,43万円,10000円,43万円,43万円,2LDK,81.27m2,https://suumo.jp/chintai/jnc_000080487072/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,賃貸アパート,プリモパッソ,東京都品川区東大井６,ＪＲ京浜東北線/大井町駅 歩6分,京急本線/立会川駅 歩11分,京急本線/鮫洲駅 歩11分,新築,3階建,2階,9.7万円,4000円,9.7万円,19.4万円,1K,27.08m2,https://suumo.jp/chintai/jnc_000086708919/?bc=...
234,賃貸アパート,プリモパッソ,東京都品川区東大井６,ＪＲ京浜東北線/大井町駅 歩6分,京急本線/立会川駅 歩11分,京急本線/鮫洲駅 歩11分,新築,3階建,3階,9.9万円,4000円,9.9万円,19.8万円,1K,28.26m2,https://suumo.jp/chintai/jnc_000086672177/?bc=...
235,賃貸アパート,プリモパッソ,東京都品川区東大井６,ＪＲ京浜東北線/大井町駅 歩6分,京急本線/立会川駅 歩11分,京急本線/鮫洲駅 歩11分,新築,3階建,3階,9.9万円,4000円,9.9万円,19.8万円,1K,28.26m2,https://suumo.jp/chintai/jnc_000086708920/?bc=...
236,賃貸マンション,Log品川サーフサイド,東京都品川区南大井３,京急本線/大森海岸駅 歩6分,ＪＲ京浜東北線/大森駅 歩6分,ＪＲ京浜東北線/大井町駅 歩29分,築7年,12階建,4階,9.2万円,10000円,-,-,1K,20.63m2,https://suumo.jp/chintai/jnc_000086561568/?bc=...


In [13]:
# 重複が削除できていなさそうだったので、改めて特定の複数列で重複削除を指示
df = df.drop_duplicates(subset=['建物名', '家賃','物件階数', '面積'])
df

,カテゴリ,建物名,住所,最寄り駅のアクセス1,最寄り駅のアクセス2,最寄り駅のアクセス3,築年数,階数情報,物件階数,家賃,管理費,礼金,敷金,間取り,面積,URL
0,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,11階,24.7万円,10000円,24.7万円,24.7万円,1LDK,43.77m2,https://suumo.jp/chintai/jnc_000086103920/?bc=...
1,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,6階,28.9万円,10000円,28.9万円,57.8万円,2LDK,54.06m2,https://suumo.jp/chintai/jnc_000085009879/?bc=...
2,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,5階,31.3万円,10000円,31.3万円,62.6万円,2LDK,57.57m2,https://suumo.jp/chintai/jnc_000085347303/?bc=...
3,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,16階,37.5万円,20000円,75万円,37.5万円,2SLDK,67.05m2,https://suumo.jp/chintai/jnc_000085227510/?bc=...
4,賃貸マンション,パークシティ大崎 ザ タワー,東京都品川区北品川５,ＪＲ山手線/大崎駅 歩8分,東急池上線/五反田駅 歩12分,京急本線/北品川駅 歩17分,築9年,地下2地上40階建,35階,43万円,10000円,43万円,43万円,2LDK,81.27m2,https://suumo.jp/chintai/jnc_000080487072/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,賃貸アパート,プリモパッソ,東京都品川区東大井６,ＪＲ京浜東北線/大井町駅 歩6分,京急本線/立会川駅 歩11分,京急本線/鮫洲駅 歩11分,新築,3階建,2階,9.6万円,4000円,9.6万円,19.2万円,1K,25.91m2,https://suumo.jp/chintai/jnc_000086672176/?bc=...
232,賃貸アパート,プリモパッソ,東京都品川区東大井６,ＪＲ京浜東北線/大井町駅 歩6分,京急本線/立会川駅 歩11分,京急本線/鮫洲駅 歩11分,新築,3階建,2階,9.7万円,4000円,9.7万円,19.4万円,1K,27.08m2,https://suumo.jp/chintai/jnc_000086672175/?bc=...
234,賃貸アパート,プリモパッソ,東京都品川区東大井６,ＪＲ京浜東北線/大井町駅 歩6分,京急本線/立会川駅 歩11分,京急本線/鮫洲駅 歩11分,新築,3階建,3階,9.9万円,4000円,9.9万円,19.8万円,1K,28.26m2,https://suumo.jp/chintai/jnc_000086672177/?bc=...
236,賃貸マンション,Log品川サーフサイド,東京都品川区南大井３,京急本線/大森海岸駅 歩6分,ＪＲ京浜東北線/大森駅 歩6分,ＪＲ京浜東北線/大井町駅 歩29分,築7年,12階建,4階,9.2万円,10000円,-,-,1K,20.63m2,https://suumo.jp/chintai/jnc_000086561568/?bc=...


In [14]:
#エクセルに落として重複がないか確認（確認をした後に非表示化）
#df.to_excel('output_file.xlsx', index=False, engine='openpyxl')

Google スプレッドシートに情報を転送

In [15]:
#必要なライブラリをインポート
import gspread
from google.auth import exceptions
from google.oauth2.service_account import Credentials

In [19]:
#Google Cloudよりキーを取得し認証情報を指定
try:
    credentials = Credentials.from_service_account_file('smooth-ocean-406910-a2bd38da5fc2.json', scopes=['https://www.googleapis.com/auth/spreadsheets'])
except exceptions.GoogleAuthError:
    credentials = Credentials.from_service_account_file('smooth-ocean-406910-a2bd38da5fc2.json', scopes=['https://www.googleapis.com/auth/spreadsheets'])

gc = gspread.authorize(credentials)

In [20]:
# 既存のスプレッドシートを開く（スプレッドシートのURLからシートを取得）
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/18IY2i7pXrhNhbW-PY1FsSiVJsBS9Qci6IVTOzWu-uT8/edit?hl=JA#gid=0'
worksheet = gc.open_by_url(spreadsheet_url).sheet1

In [21]:
# データフレームをGoogleスプレッドシートに書き込む
worksheet.clear()  # シートをクリア
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

C:\Users\masay\AppData\Local\Temp\ipykernel_2520\4288850697.py:3: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '18IY2i7pXrhNhbW-PY1FsSiVJsBS9Qci6IVTOzWu-uT8',
 'updatedRange': "'シート1'!A1:P191",
 'updatedRows': 191,
 'updatedColumns': 16,
 'updatedCells': 3056}